In [41]:
import pandas as pd
import datetime
import plotly.express as px


In [42]:
basepath = r'C:\Users\Joyqiao\Documents\CMU\RA\HIGHMARK Trajectory claim\HIghmark Project'

earlist = pd.read_csv(basepath+'\\Clinical Data\\Clinical Data\\egfr_earliest.csv',index_col = 0)

In [43]:
earlist = earlist.sort_values(by = ['TMA_Acct','min']).drop_duplicates(subset = ['TMA_Acct','egfr','stage'],keep = 'first')

earlist.to_csv(basepath+'\\Clinical Data\\Clinical Data\\egfr_earliest.csv')

In [44]:
transplant = pd.read_csv(basepath+'\\data\\sorted_data\\transplant_patient_clean.csv',index_col = 0)

In [45]:
dialysis = pd.read_csv(basepath+'\\data\\sorted_data\\dialysis_patient_clean.csv',index_col = 0)

In [46]:
dialysis

,0,1,2
29447,503,1756,V4511
32911,506,1020,V4511
34977,552,14,V4511
39898,586,1029,V4511
42158,601,1377,V4511
...,...,...,...
558462,50621,2584,Z992
614413,50838,3377,Z992
645764,50449,2903,Z992
646848,50534,2179,Z992


In [47]:
transplant

,0,1,2
67933,601,2277,652
727779,4167,2115,652
805141,4429,1207,652
1543,4916,3333,652
101687,5570,610,652
...,...,...,...
373921,47207,938,652
192185,49073,3269,652
206016,49116,2976,652
258031,49249,3403,652


In [48]:
def see_duration(data):
    '''find how long it takes for transplant/dialysis patient to progress from their initial stage to dialysis/transplant
    input:dialysis/transplant patient
    output list of progression time by initial stage'''
    data.rename({'0':'TMA_Acct','1':'Days','2':'Code'},axis =1 ,inplace = True)
    data ['identified_date'] = pd.to_datetime('2010-01-01') + data['Days'].apply(lambda i:pd.Timedelta(days = i))   
    merged_data = data.merge(earlist,on = 'TMA_Acct',how = 'left')

    merged_data['min'] = pd.to_datetime(merged_data['min'])
    merged_data['duration'] =( merged_data['identified_date'] - merged_data['min']).dt.components['days']
    merged_data['stage'] = merged_data['stage'].fillna('Unknown')
    
    fig4 = px.histogram(data_frame=merged_data,x='duration',facet_col= 'stage',color = 'stage',category_orders={"stage": ['Unknown','1','2','3a','3b','4','5']})
    fig4.show()
    fig5 = px.box(data_frame=merged_data,y='duration',x= 'stage',color = 'stage',category_orders={"stage": ['Unknown','1','2','3a','3b','4','5']})
    fig5.show()
    
    
    return merged_data
    
    
    
    

In [49]:
merged_data_t = see_duration(transplant)


In [50]:
basepath = r'C:\Users\Joyqiao\Documents\CMU\RA\HIGHMARK Trajectory claim\HIghmark Project'
import os
os.chdir(basepath+'\data\\cleaned_data')
max_l = pd.read_csv('max_min_patient.csv')['max']
def find_tma(tma):
    find = False
    i = 0
    while find == False:
        tma_max = max_l[i]
        if tma <= tma_max:
            data = pd.read_csv('cleaned_{}.csv'.format(i))

            if tma in data['TMA_Acct']:
                records= data[data['TMA_Acct']==tma]
                find = True
            else:
                data = pd.read_csv('cleaned_16.csv')
                records= data[data['TMA_Acct']==tma]
                find = True
        else:
            i+=1
    return records

In [51]:
egfr = pd.read_csv(basepath+'\\Clinical Data\\Clinical Data\\All_egfr.csv')

In [52]:
egfr

,Unnamed: 0,TMA_Acct,date,egfr,source,stage
0,16,9,2000-09-22,43.651041,TMA_Lab_Serum,3b
1,11,9,2000-11-15,47.448244,TMA_Lab_Serum,3a
2,2,9,2001-05-21,47.116106,TMA_Lab_Serum,3a
3,6,9,2001-10-30,47.116106,TMA_Lab_Serum,3a
4,12,9,2002-03-14,51.171709,TMA_Lab_Serum,3a
...,...,...,...,...,...,...
103308,8393,51580,2018-12-21 00:00:00,37.000000,TMA_Lab_Egfr,3b
103309,8592,52742,2018-01-30 00:00:00,45.000000,TMA_Lab_Egfr,3a
103310,9110,52742,2018-03-01 00:00:00,51.000000,TMA_Lab_Egfr,3a
103311,9805,52742,2018-09-20 00:00:00,45.000000,TMA_Lab_Egfr,3a


In [53]:
merged_data_t[merged_data_t['stage'] =='1']

,TMA_Acct,Days,Code,identified_date,egfr,stage,min,duration
47,31925,2078,92,2015-09-10,103.295916,1,2010-05-11,1948.0
64,47019,2756,652,2017-07-19,138.000000,1,2012-09-19,1764.0


In [65]:
pd.set_option('display.max_columns',None)
for i in merged_data_t[merged_data_t['stage'] =='1']['TMA_Acct']:
    
    egfr_target = egfr[egfr['TMA_Acct'] == i]
    fig = px.scatter(egfr_target,x = 'date',y = 'egfr',hover_data = ['source'])
    fig.show()
    target = find_tma(i)
    target['date'] = pd.to_datetime('2010-01-01') + target['icrd_dt_deidentified'].apply(lambda i:pd.Timedelta(days = i))   
    fig = px.scatter(target[target['claim_type'] == 'I'],x = 'date',y = 'eacl_prv_alcrg_at',hover_data = ['claim_type'])
    display(target)
    fig.show()
    

C:\Users\Joyqiao\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning:

Columns (10,12,13,14,15,16,17,18,19,20,21,23,25,39,40,41,42,43,44,45,46,47,48,49,52,53) have mixed types.Specify dtype option on import or set low_memory=False.



,Unnamed: 0,Unnamed: 0.1,claim_type,line_number,place_service,eacl_prv_alcrg_at,mmn_mr_hcg_catg_cd,proc_code,proc_code_format,prin_proc_mod,ip_prn_proc_code,ip_prn_proc_format_cd,ip_other_proc1,ip_other_proc2,ip_other_proc3,ip_other_proc4,ip_other_proc5,ip_other_proc6,ip_other_proc7,ip_other_proc8,ip_other_proc9,ip_other_proc10,pri_diag_cd,prim_diag_qfr_cd,admitting_diag_cd,admi_diag_qfr_cd,dx1,dx2,dx3,dx4,dx5,dx6,dx7,dx8,dx9,dx10,dx11,dx12,dx13,dx14,dx15,dx16,dx17,dx18,dx19,dx20,dx21,dx22,dx23,dx24,dx25,dx26,diag_qfr_cd_001,admission_source,admission_type,discharge_status,current_drg,bill_provider_specialty,TMA_Acct,claim_number_deidentified,provider_bill_npi_deidentified,icrd_dt_deidentified,admission_date_deidentified,discharge_date_deidentified,date
204447,208540,7095001,O,3,22,106.04,O41b,36430,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2859,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,1.0,NaN,301.0,31925.0,4466910,45637,1639,NaN,NaN,2014-06-28
204448,208541,7095004,O,3,22,12.36,O15,85025,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23875,9,NaN,NaN,2859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,1,3.0,1.0,NaN,302.0,31925.0,3627998,15009,1624,NaN,NaN,2014-06-13
204449,208542,7095005,O,3,22,12.36,O15,85025,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23875,9,NaN,NaN,2859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,1,3.0,1.0,NaN,302.0,31925.0,5112089,15009,1638,NaN,NaN,2014-06-27
204450,208543,7095006,O,3,22,12.36,O15,85025,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23875,9,NaN,NaN,2859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,1,3.0,1.0,NaN,302.0,31925.0,5286343,15009,1617,NaN,NaN,2014-06-06
204451,208544,7095007,O,3,22,12.36,O15,85025,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23875,9,NaN,NaN,2859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,1,3.0,1.0,NaN,302.0,31925.0,5583310,15009,1630,NaN,NaN,2014-06-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636170,647150,7097486,O,26,22,80.61,O12a,88271,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23875,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,1.0,NaN,302.0,31925.0,129064,15009,1874,NaN,NaN,2015-02-18
636171,647151,7097582,O,9,22,33.55,O13b,71020,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23875,9,NaN,NaN,V7283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,1,3.0,1.0,NaN,301.0,31925.0,4207791,45981,2063,NaN,NaN,2015-08-26
636172,647152,7097610,O,7,22,10.40,O15,85027,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79439,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,1.0,NaN,301.0,31925.0,6040205,45637,2055,NaN,NaN,2015-08-18
636173,647153,7097883,O,1,BF,30.76,O12b,20610,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71941,9,NaN,NaN,71591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,2,9.0,1.0,NaN,326.0,31925.0,116952,27917,1707,NaN,NaN,2014-09-04


C:\Users\Joyqiao\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning:

Columns (13,14,15,16,17,18,19,20,21,23,25,41,42,43,44,45,46,47,48,49,52,53) have mixed types.Specify dtype option on import or set low_memory=False.



,Unnamed: 0,Unnamed: 0.1,claim_type,line_number,place_service,eacl_prv_alcrg_at,mmn_mr_hcg_catg_cd,proc_code,proc_code_format,prin_proc_mod,ip_prn_proc_code,ip_prn_proc_format_cd,ip_other_proc1,ip_other_proc2,ip_other_proc3,ip_other_proc4,ip_other_proc5,ip_other_proc6,ip_other_proc7,ip_other_proc8,ip_other_proc9,ip_other_proc10,pri_diag_cd,prim_diag_qfr_cd,admitting_diag_cd,admi_diag_qfr_cd,dx1,dx2,dx3,dx4,dx5,dx6,dx7,dx8,dx9,dx10,dx11,dx12,dx13,dx14,dx15,dx16,dx17,dx18,dx19,dx20,dx21,dx22,dx23,dx24,dx25,dx26,diag_qfr_cd_001,admission_source,admission_type,discharge_status,current_drg,bill_provider_specialty,TMA_Acct,claim_number_deidentified,provider_bill_npi_deidentified,icrd_dt_deidentified,admission_date_deidentified,discharge_date_deidentified,date
189773,196288,13852827,O,5,22,3.65,O15,82247,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V427,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,1.0,NaN,302.0,47019.0,1043338,48397,1282,NaN,NaN,2013-07-06
189774,196289,13852828,O,5,22,248.37,O11,99284,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2752,9,NaN,NaN,72981,V426,V427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,1,1.0,1.0,NaN,334.0,47019.0,456288,29021,1320,NaN,NaN,2013-08-13
189775,196290,13852829,O,5,22,171.00,O15,80197,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V426,9,NaN,NaN,V5869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,1,3.0,1.0,NaN,302.0,47019.0,2852207,50060,1313,NaN,NaN,2013-08-06
189776,196291,13852830,O,5,22,24.45,O15,82977,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V426,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,1.0,NaN,334.0,47019.0,1147297,29021,1320,NaN,NaN,2013-08-13
189777,196292,13852831,O,5,22,20.92,O15,82247,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78605,9,NaN,NaN,7862,78791,27700,V426,V427,V4585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,2,3.0,1.0,NaN,302.0,47019.0,4062605,45210,1400,NaN,NaN,2013-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582677,615420,13862543,O,13,22,153.80,O13b,71034,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5194,9,NaN,NaN,V426,99684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,1,3.0,1.0,NaN,301.0,47019.0,1174113,45981,1652,NaN,NaN,2014-07-11
582678,615421,13862579,O,2,22,148.15,O41j,94620,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99684,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,1.0,NaN,301.0,47019.0,3494669,45981,1771,NaN,NaN,2014-11-07
582679,615422,13862584,O,3,22,49.33,O41d,99214,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99684,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,1.0,NaN,301.0,47019.0,3494669,45981,1771,NaN,NaN,2014-11-07
582680,615423,13862670,O,5,22,58.64,O41d,99204,H,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99684,9,NaN,NaN,27700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,1,3.0,1.0,NaN,301.0,47019.0,1264266,45981,1699,NaN,NaN,2014-08-27


In [55]:
merged_data_t[merged_data_t['stage'] =='2']

,TMA_Acct,Days,Code,identified_date,egfr,stage,min,duration
28,21012,183,652,2010-07-03,81.053517,2,2010-07-08,-5.0
44,31070,2451,652,2016-09-17,62.457261,2,2011-01-31,2056.0
55,37703,1713,93,2014-09-10,69.000000,2,2013-06-25,442.0
56,38754,2640,8,2017-03-25,78.187260,2,2013-12-21,1190.0
67,49116,2976,652,2018-02-24,60.000000,2,2010-02-02,2944.0


In [62]:
for i in merged_data_t[merged_data_t['stage'] =='2']['TMA_Acct']:
    egfr_target = egfr[egfr['TMA_Acct'] == i]
    fig = px.scatter(egfr_target,x = 'date',y = 'egfr',hover_data = ['source'])
    fig.show()
    target = find_tma(i)
    target['date'] = pd.to_datetime('2010-01-01') + target['icrd_dt_deidentified'].apply(lambda i:pd.Timedelta(days = i))   
    fig = px.scatter(target[target['claim_type'] == 'I'],x = 'date',y = 'eacl_prv_alcrg_at',hover_data = ['claim_type'])
    fig.show()
    

C:\Users\Joyqiao\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning:

Columns (10,12,13,14,15,16,17,18,19,20,21,23,25,37,38,40,41,42,43,44,45,46,47,48,49,52,53) have mixed types.Specify dtype option on import or set low_memory=False.



C:\Users\Joyqiao\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning:

Columns (10,12,13,14,15,16,17,18,19,20,21,23,25,36,37,38,39,40,41,42,43,44,45,46,47,48,49,52,53) have mixed types.Specify dtype option on import or set low_memory=False.



C:\Users\Joyqiao\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning:

Columns (10,12,13,14,15,16,17,18,19,20,21,23,25,40,41,42,43,44,45,46,47,48,49,52,53) have mixed types.Specify dtype option on import or set low_memory=False.



C:\Users\Joyqiao\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning:

Columns (10,12,13,14,15,16,17,18,19,20,21,23,25,39,40,41,42,43,44,45,46,47,48,49,52,53) have mixed types.Specify dtype option on import or set low_memory=False.



C:\Users\Joyqiao\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning:

Columns (11,13,14,15,16,17,18,19,20,21,23,25,40,41,42,43,44,45,46,47,48,49,52,53) have mixed types.Specify dtype option on import or set low_memory=False.



In [ ]:
WHERE IS ALL THE TRANSPLANT AND DIALYSIS PAITEINT FALL INTO CLUSTERS

In [57]:
help(px.scatter)

Help on function scatter in module plotly.express._chart_types:

scatter(data_frame=None, x=None, y=None, color=None, symbol=None, size=None, hover_name=None, hover_data=None, custom_data=None, text=None, facet_row=None, facet_col=None, facet_col_wrap=0, facet_row_spacing=None, facet_col_spacing=None, error_x=None, error_x_minus=None, error_y=None, error_y_minus=None, animation_frame=None, animation_group=None, category_orders={}, labels={}, orientation=None, color_discrete_sequence=None, color_discrete_map={}, color_continuous_scale=None, range_color=None, color_continuous_midpoint=None, symbol_sequence=None, symbol_map={}, opacity=None, size_max=None, marginal_x=None, marginal_y=None, trendline=None, trendline_color_override=None, log_x=False, log_y=False, range_x=None, range_y=None, render_mode='auto', title=None, template=None, width=None, height=None)
        In a scatter plot, each row of `data_frame` is represented by a symbol
        mark in 2D space.
        
    Parameters
  

In [58]:
ls

 Volume in drive C is Windows
 Volume Serial Number is BA8F-7F5E

 Directory of C:\Users\Joyqiao\Documents\CMU\RA\HIGHMARK Trajectory claim\HIghmark Project\data\cleaned_data

11/15/2020  09:51 PM    <DIR>          .
11/15/2020  09:51 PM    <DIR>          ..
11/15/2020  02:34 PM        69,888,389 cleaned_0.csv
11/15/2020  02:37 PM        66,417,182 cleaned_1.csv
11/15/2020  03:16 PM        82,197,627 cleaned_10.csv
11/15/2020  03:22 PM        79,762,181 cleaned_11.csv
11/15/2020  03:27 PM        85,288,939 cleaned_12.csv
11/15/2020  03:32 PM        86,903,824 cleaned_13.csv
11/15/2020  03:37 PM        82,574,530 cleaned_14.csv
11/15/2020  03:41 PM        82,458,208 cleaned_15.csv
11/15/2020  03:44 PM        82,458,208 cleaned_16.csv
11/15/2020  02:41 PM        72,044,443 cleaned_2.csv
11/15/2020  02:45 PM        66,387,522 cleaned_3.csv
11/15/2020  02:48 PM        57,601,269 cleaned_4.csv
11/15/2020  02:53 PM        75,899,289 cleaned_5.csv
11/15/2020  02:56 PM        76,540,969 cleane

In [59]:

merged_data = see_duration(dialysis)


In [60]:
merged_data['duration'].sort_values()

68    -2333.0
818   -2052.0
24    -1979.0
339   -1951.0
819   -1738.0
        ...  
922       NaN
928       NaN
935       NaN
936       NaN
937       NaN
Name: duration, Length: 938, dtype: float64